<a href="https://colab.research.google.com/github/hamedatasi/enc/blob/main/nb/Gemma3_(4B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [55]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

==((====))==  Unsloth 2025.3.14: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


We now add LoRA adapters so we only need to update a small amount of parameters!

In [56]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 16,           # Larger = higher accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [58]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [59]:
from datasets import load_dataset
from google.colab import drive
import os
from datasets import load_dataset

# Mount Google Drive
drive.mount('/content/drive')

# Define the correct path to your file in Google Drive
file_path = "/content/drive/My Drive/ai/gemma_errors.jsonl"  # Change if your file is in a subfolder

# Verify if the file exists
if os.path.exists(file_path):
    print(f"✅ Found file: {file_path}")
else:
    raise FileNotFoundError(f"❌ File not found at {file_path}. Please check Google Drive.")

# Load dataset
dataset = load_dataset("json", data_files=file_path, split="train")
print("✅ Dataset Loaded Successfully!")
print("📝 Available Columns:", dataset.column_names)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Found file: /content/drive/My Drive/ai/gemma_errors.jsonl
✅ Dataset Loaded Successfully!
📝 Available Columns: ['conversations']


We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [60]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Let's see how row 100 looks like!

In [61]:
dataset[50]

{'conversations': [{'role': 'system',
   'content': 'You are a helpful AI assistant.'},
  {'role': 'user',
   'content': 'در ارسال به مودیان تکروسیتم خطای:  صورتحساب بارنامه'},
  {'role': 'assistant',
   'content': 'کد ملی فرستنده و گیرنده را حتما باید مقداردهی کنید - اگر نوع بارنامه به صورت جاده ابی باشد کد ملی راننده اجباری است- شماره ناوگان از طریق ماشین انتخاب می شود و حتما اجباری است - کشور مبدا و مقصد الزامی و اجباری است- اگر کشور ایران باشد  انتخاب شهر الزامی است-نوع حمل به عنوان کالا شما انتخاب می شود و اجباری است - موضوع حمل به عنوان کالا های حمل شده انتخاب می شود و اجباری است - در بارنامه مقدار حتما باید برابر یک باشد و تناژ به عنوان وزن ارسال می شود  از سایت   https://www.takrosystem.ir/blog/3/how-to-send-a-bill-of-lading-to-the-taxpayer-system-75  می توانید اموزش بارنامه را دریافت کنید'}]}

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`

In [62]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True)

Let's see how the chat template did! Notice `Gemma-3` default adds a `<bos>`!

In [64]:
dataset[50]["text"]

'<bos><start_of_turn>user\nYou are a helpful AI assistant.\n\nدر ارسال به مودیان تکروسیتم خطای:  صورتحساب بارنامه<end_of_turn>\n<start_of_turn>model\nکد ملی فرستنده و گیرنده را حتما باید مقداردهی کنید - اگر نوع بارنامه به صورت جاده ابی باشد کد ملی راننده اجباری است- شماره ناوگان از طریق ماشین انتخاب می شود و حتما اجباری است - کشور مبدا و مقصد الزامی و اجباری است- اگر کشور ایران باشد  انتخاب شهر الزامی است-نوع حمل به عنوان کالا شما انتخاب می شود و اجباری است - موضوع حمل به عنوان کالا های حمل شده انتخاب می شود و اجباری است - در بارنامه مقدار حتما باید برابر یک باشد و تناژ به عنوان وزن ارسال می شود  از سایت   https://www.takrosystem.ir/blog/3/how-to-send-a-bill-of-lading-to-the-taxpayer-system-75  می توانید اموزش بارنامه را دریافت کنید<end_of_turn>\n'

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [65]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 600,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16
Unsloth: We found double BOS tokens - we shall remove one automatically.


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [66]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Let's verify masking the instruction part is done! Let's print the 100th row again:

In [67]:
tokenizer.decode(trainer.train_dataset[50]["input_ids"])

'<bos><start_of_turn>user\nYou are a helpful AI assistant.\n\nدر ارسال به مودیان تکروسیتم خطای:  صورتحساب بارنامه<end_of_turn>\n<start_of_turn>model\nکد ملی فرستنده و گیرنده را حتما باید مقداردهی کنید - اگر نوع بارنامه به صورت جاده ابی باشد کد ملی راننده اجباری است- شماره ناوگان از طریق ماشین انتخاب می شود و حتما اجباری است - کشور مبدا و مقصد الزامی و اجباری است- اگر کشور ایران باشد  انتخاب شهر الزامی است-نوع حمل به عنوان کالا شما انتخاب می شود و اجباری است - موضوع حمل به عنوان کالا های حمل شده انتخاب می شود و اجباری است - در بارنامه مقدار حتما باید برابر یک باشد و تناژ به عنوان وزن ارسال می شود  از سایت   https://www.takrosystem.ir/blog/3/how-to-send-a-bill-of-lading-to-the-taxpayer-system-75  می توانید اموزش بارنامه را دریافت کنید<end_of_turn>\n'

Now let's print the masked out example - you should see only the answer is present:

In [68]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[50]["labels"]]).replace(tokenizer.pad_token, " ")

'                                     کد ملی فرستنده و گیرنده را حتما باید مقداردهی کنید - اگر نوع بارنامه به صورت جاده ابی باشد کد ملی راننده اجباری است- شماره ناوگان از طریق ماشین انتخاب می شود و حتما اجباری است - کشور مبدا و مقصد الزامی و اجباری است- اگر کشور ایران باشد  انتخاب شهر الزامی است-نوع حمل به عنوان کالا شما انتخاب می شود و اجباری است - موضوع حمل به عنوان کالا های حمل شده انتخاب می شود و اجباری است - در بارنامه مقدار حتما باید برابر یک باشد و تناژ به عنوان وزن ارسال می شود  از سایت   https://www.takrosystem.ir/blog/3/how-to-send-a-bill-of-lading-to-the-taxpayer-system-75  می توانید اموزش بارنامه را دریافت کنید<end_of_turn>\n'

In [69]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
9.693 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 74 | Num Epochs = 67 | Total steps = 600
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,802,496/4,000,000,000 (0.75% trained)


Step,Training Loss
1,4.008500
2,4.006400
3,3.786400
4,4.096700
5,3.842300
6,4.063800
7,3.960800
8,4.354200
9,3.674100
10,3.765600


In [37]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2269.6132 seconds used for training.
37.83 minutes used for training.
Peak reserved memory = 9.693 GB.
Peak reserved memory for training = 0.547 GB.
Peak reserved memory % of max memory = 65.755 %.
Peak reserved memory for training % of max memory = 3.711 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [38]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\n13, 21, 34, 55, 89,...\n<end_of_turn>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [39]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The sky is blue due to a phenomenon called **Rayleigh scattering**. Here's a breakdown of how it works:

1. **Sunlight and Colors:** Sunlight is actually made up of all the colors of the rainbow. You can see this when light passes through a prism or when you see a rainbow.




<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [41]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
#model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
#tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define save path in Google Drive
save_path = "/content/drive/My Drive/gemma-3"

# Ensure the directory exists
os.makedirs(save_path, exist_ok=True)
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ Model and tokenizer saved to Google Drive at {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model and tokenizer saved to Google Drive at /content/drive/My Drive/gemma-3


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [50]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "خطای مودیان تکرو سیستم ",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 500, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

خطای مودیان تکرو سیستم (Multi-factor authentication) معمولا به دلیل موارد زیر است:
*   **عدم تطابق شناسه مودیان:** اگر شناسه مودیان در سیستم مودیان با شناسه مودیان در سیستم مودیان ماتیون (یا هر سیستم دیگر که مودیان به آن متصل است) یکسان نباشد یا مودیان شناسه خود را تغییر داده باشد این خطا را خواهد داشت
*   **عدم تطابق زمان:** اگر زمان سیستم مودیان با زمان سیستم مودیان ماتیون یکسان نباشد این خطا را خواهد داشت
*   **عدم تطابق نسخه نرم افزار:** اگر نسخه نرم افزار مودیان ماتیون با نسخه نرم افزار مودیان ماتیون یکسان نباشد این خطا را خواهد داشت
*   **عدم تطابق فایروال:** اگر فایروال بین سیستم مودیان ماتیون و سیستم مودیان ماتیون مانع از برقراری ارتباط شود این خطا را خواهد داشت<end_of_turn>


### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [49]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
